## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

## Collecting Dataset

In [2]:
df = pd.read_csv("autos.csv")

In [3]:
df

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3T�RER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


## Read and clean data sets

In [4]:
df.seller.value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

### Removing the seller type having only least cars

In [5]:
df=df[df.seller != 'gewerblich']

### Now all the sellers are same so we can get rid of this column 

In [6]:
df=df.drop('seller',1)

In [7]:
df.offerType.value_counts()

Angebot    371513
Gesuch         12
Name: offerType, dtype: int64

### Removing the Offer Type that is having only 12 listings 

In [8]:
df=df[df.offerType != 'Gesuch']

In [9]:
df

,dateCrawled,name,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3T�RER,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


### Now all the offers are same so we can get rid of this column 

In [10]:
df=df.drop('offerType',1)

In [11]:
df.shape

(371513, 18)

### Updating data to the present trend

In [12]:
df = df[(df.powerPS > 50) & (df.powerPS <900)]

In [13]:
df.shape

(319704, 18)

In [14]:
df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]

In [15]:
df.shape

(309166, 18)

In [16]:
df.head()

,dateCrawled,name,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3T�RER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07


In [17]:
df.columns

Index(['dateCrawled', 'name', 'price', 'abtest', 'vehicleType',
       'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'kilometer',
       'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage',
       'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen'],
      dtype='object')

### Dropping unnecessary columns

In [18]:
df.drop(['name', 'abtest', 'dateCrawled','nrOfPictures', 'lastSeen', 'postalCode', 'dateCreated'], axis='columns', inplace=True)

In [19]:
df

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,650,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja
...,...,...,...,...,...,...,...,...,...,...,...
371520,3200,limousine,2004,manuell,225,leon,150000,5,benzin,seat,ja
371524,1199,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


In [20]:
new_df = df.copy()

In [21]:
new_df.columns

Index(['price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS',
       'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage'],
      dtype='object')

In [22]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,650,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja


### Dropping Dupilcates

In [23]:
new_df = new_df.drop_duplicates(['price', 'vehicleType', 'yearOfRegistration','gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType' , 'notRepairedDamage'])

In [24]:
new_df.shape

(285140, 11)

### Translating Data in understandable format

In [25]:
new_df.gearbox.replace(('manuell', 'automatik'), ('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin', 'andere', 'elektro'), ('petrol', 'others', 'electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio', 'kombi', 'andere'),('small car','convertible', 'combination', 'others'), inplace=True)
new_df.notRepairedDamage.replace(('ja', 'nein'), ('Yes', 'No'), inplace =True)

In [26]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manual,190,NaN,125000,5,diesel,audi,Yes
2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,NaN
3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No
4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No
5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes


In [27]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [28]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,coupe,2011,manual,190,not-declared,125000,5,diesel,audi,Yes
2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,not-declared
3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No
4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No
5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes


### Creating new preprocessed Data

In [29]:
new_df.to_csv("pp_data.csv")

In [30]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(new_df[i])
    tr = mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    new_df.loc[:, i+'_labels'] = pd.Series(tr, index=new_df.index)
    
labeled = new_df[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


### Split the data into dependent and independent variables.

In [31]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

In [32]:
X

array([[  2011,    190, 125000, ...,      1,      1,      3],
       [  2004,    163, 125000, ...,     14,      1,      8],
       [  2001,     75, 150000, ...,     38,      7,      7],
       ...,
       [  1996,    102, 150000, ...,     38,      1,      0],
       [  2002,    100, 150000, ...,     38,      1,      1],
       [  2013,    320,  50000, ...,      2,      7,      4]], dtype=int64)

In [33]:
Y

array([18300,  9800,  1500, ...,  9200,  3400, 28990], dtype=int64)

In [34]:
Y = Y.reshape(-1,1)

In [35]:
print(X.shape,Y.shape)

(278575, 10) (278575, 1)


### Train test split

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X ,Y ,test_size = 0.3,random_state = 3)

In [37]:
X_train

array([[  2009,     90, 150000, ...,     10,      1,      0],
       [  2006,    150, 150000, ...,      2,      1,      4],
       [  1999,    102, 150000, ...,     20,      7,      4],
       ...,
       [  1994,    102, 150000, ...,      2,      7,      4],
       [  1997,    170, 125000, ...,     20,      7,      4],
       [  2012,    313,  50000, ...,      1,      1,      1]], dtype=int64)

### Choosing appropriate model

### Multilinear Regression

In [38]:
multiple_lin_reg = LinearRegression()
multiple_lin_reg.fit(X_train,Y_train)

LinearRegression()

In [39]:
y_pred_mlr = multiple_lin_reg.predict(X_test)

### Metrics Evaluation

In [40]:
mae = mean_absolute_error(Y_test, y_pred_mlr)
mse = mean_squared_error(Y_test, y_pred_mlr)
rmse = np.sqrt(mse)
rmsle = np.log(rmse)
n,k = X_train.shape
r2=r2_score(Y_test,y_pred_mlr)
adj_r2= 1 - ((1-r2)*(n-1)/(n-k-1))
print(mae,mse,rmse,rmsle,r2,adj_r2)

3137.89414643729 29434090.807211053 5425.3194198324445 8.598832055907133 0.5759870208048179 0.5759652755458473


In [41]:
regressor = RandomForestRegressor(n_estimators =1000,max_depth=10,random_state=34)

In [42]:
regressor.fit(X_train,np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [43]:
y_pred = regressor.predict(X_test)

### Metrics Evaluation

In [44]:
mae = mean_absolute_error(Y_test, y_pred)
mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
rmsle = np.log(rmse)
n,k = X_train.shape
r2=r2_score(Y_test,y_pred)
adj_r2= 1 - ((1-r2)*(n-1)/(n-k-1))
print(mae,mse,rmse,rmsle,r2,adj_r2)

1624.2919043250051 10729873.982609846 3275.648635401826 8.094271185069976 0.8454307332421266 0.8454228062472008


#### R^2 score is an indicator of accuracy of Regression Models, and the accuracy is measured as close to 1 of this value. Therefore, as seen, Random Forest Regression is better than Multiple Linear Regression Model on this dataset when comparing their R^2 scores.

### Save the model

In [45]:
filename = 'crvp.sav'
pickle.dump(regressor,open(filename,'wb'))